In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from transformers import AutoTokenizer, BertModel
from tqdm import tqdm
from data_utils import *
import fasttext, re, json, faiss
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import average_precision_score
import matplotlib.pyplot as plt
from model import Net
import pickle

%load_ext autoreload
%autoreload 2

In [2]:
from transformers import AutoModelForCausalLM

model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id, use_flash_attention_2=True, device_map="auto")

messages = [
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
]

input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")

outputs = model.generate(input_ids, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

model.safetensors.index.json:   0%|          | 0.00/92.7k [00:00<?, ?B/s]

c:\Users\Luis\miniconda3\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Luis\.cache\huggingface\hub\models--mistralai--Mixtral-8x7B-Instruct-v0.1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model-00001-of-00019.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

model-00002-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

In [ ]:
config = {
    'IMG_WIDTH': 224,
    'IMG_HEIGHT': 224,
    'TEST_DATASET_DIR': '../Week 4/data/val2014',
    'batch_size': 16,
    'n_neighbors': 5,
    'classifier': 'knn',
    'text_encoder_type': 'bert',
    'mode': "img2text",
    'metric': 'euclidean',
    'voronoi_cells': 64,
    'lookup_cells': 8,
    'device': torch.device("cuda" if torch.cuda.is_available() else "cpu")
}

In [ ]:
# File containing images info (file_name)
with open(f"data/captions_val2014.json", 'r') as f:
    captions_val = json.load(f)

# Dataset for the retrieval
retrieval_dataset = CocoMetricDataset(
    root=config["TEST_DATASET_DIR"],
    captions_file=captions_val,
    transforms=CustomTransform(config, mode="train"))

dataloader = DataLoader(retrieval_dataset, batch_size=config['batch_size'], shuffle=False, collate_fn=coco_collator)

In [ ]:
model = Net(config['text_encoder_type'])
model.load_state_dict(torch.load("pretrained/image2text_hard_ft.pth", map_location=config['device']))
model = model.to(config['device'])

In [ ]:
def extract_features(loader, model, device, mode=None):
    model.eval()
    with torch.no_grad():
        db_features, query_features, labels = [], [], []
        for img, captions, label in tqdm(loader, total=len(loader), desc='Extracting features'):
            img_f, text_f = model(img.to(device), captions)
            if mode == "img2text":
                db_features.append(text_f.cpu().numpy())
                query_features.append(img_f.cpu().numpy())
            else:
                db_features.append(img_f.cpu().numpy())
                query_features.append(text_f.cpu().numpy())

            labels.append(label)
        return np.concatenate(db_features).astype('float32'), np.concatenate(query_features).astype('float32'), np.concatenate(labels)

# k-NN Classifier for Image Retrieval
class ImageRetrievalSystem:
    def __init__(self, model, database_loader, config):
        self.model = model
        self.database_loader = database_loader
        self.device = config['device']
        self.mode = config['mode']

        self.dim = 2048
        self.classifier_type = config['classifier']
        self.n_neighbors = config['n_neighbors']

        if self.classifier_type == 'knn':
            self.classifier = NearestNeighbors(n_neighbors=config['n_neighbors'], metric=config['metric'])
        else:
            self.classifier = faiss.IndexIVFFlat(faiss.IndexFlatL2(self.dim), self.dim, config['voronoi_cells'])
            self.classifier.nprobe = config['lookup_cells']

    def fit_and_retrieve(self):

        db_features, query_features, labels = extract_features(self.database_loader, self.model, self.device, mode=self.mode)
        
        if self.classifier_type == 'knn':
            self.classifier.fit(db_features)
            _, predictions = self.classifier.kneighbors(query_features, return_distance=True)
        else:
            self.classifier.train(db_features)
            self.classifier.add(db_features)
            _, predictions = self.classifier.search(query_features, self.n_neighbors)

        return predictions, labels   

In [ ]:
def average_precision(preds, true_label):
    """Calculate average precision for a single class/query."""
    relevant = np.cumsum([1 if p == true_label else 0 for p in preds])
    precision_at_k = relevant / np.arange(1, len(preds) + 1)
    return np.sum(precision_at_k * (relevant > 0)) / np.maximum(np.sum(relevant > 0), 1)

def evaluate(predictions, labels):
    # Prec@1
    prec_at_1 = np.mean([1 if predictions[i, 0] == labels[i] else 0 for i in range(predictions.shape[0])])

    # Prec@5
    prec_at_5 = np.mean([np.any([1 if predictions[i, j] == labels[i] else 0 for j in range(5)]) for i in range(predictions.shape[0])])

    ap_scores = [average_precision(predictions[i], labels[i]) for i in range(predictions.shape[0])]
    mean_ap = np.mean(ap_scores)

    return prec_at_1, prec_at_5, mean_ap

In [ ]:
pipeline = ImageRetrievalSystem(model, dataloader, config)
predictions_idx, labels = pipeline.fit_and_retrieve()

predictions = labels[predictions_idx]

prec_at_1, prec_at_5, mean_ap = evaluate(predictions, labels)

print(f'Precision at 1: {prec_at_1:.4f}')
print(f'Precision at 5: {prec_at_5:.4f}')
print(f'Mean Average Precision: {mean_ap:.4f}')

In [ ]:
# Function to denormalize image
def denormalize(normalized_img):
    mean = torch.tensor([0.485, 0.456, 0.406])
    std = torch.tensor([0.229, 0.224, 0.225])
    denormalized_img = normalized_img * std[:, None, None] + mean[:, None, None]
    return denormalized_img

In [ ]:
n = 3
retrieval_vis_data = [retrieval_dataset[x] for x in np.unique(predictions_idx[:n].flatten())]
gt_vis_data = [retrieval_dataset[x] for x in range(n)]

retrieved_embeddings = []
for img, text, _ in retrieval_vis_data:
    with torch.no_grad():
        img_f, text_f = model(img.unsqueeze(0).to(config['device']), [text])
    if config['mode'] == "img2text":
        retrieved_embeddings.append(text_f.cpu().numpy())
    else:
        retrieved_embeddings.append(img_f.cpu().numpy())

gt_embeddings, query_embeddings = [], []
for img, text, _ in gt_vis_data:
    with torch.no_grad():
        img_f, text_f = model(img.unsqueeze(0).to(config['device']), [text])
    if config['mode'] == "img2text":
        gt_embeddings.append(text_f.cpu().numpy())
        query_embeddings.append(img_f.cpu().numpy())
    else:
        gt_embeddings.append(img_f.cpu().numpy())
        query_embeddings.append(text_f.cpu().numpy())

retrieved_embeddings = np.concatenate(retrieved_embeddings).astype('float32')
query_embeddings = np.concatenate(query_embeddings).astype('float32')
gt_embeddings = np.concatenate(gt_embeddings).astype('float32')

In [ ]:
from matplotlib.offsetbox import OffsetImage, AnnotationBbox, TextArea
import textwrap
from sklearn.manifold import TSNE
from imojify import imojify

textlength = 50

# Combine both sets of embeddings for UMAP
combined_embeddings = np.vstack([query_embeddings, retrieved_embeddings, gt_embeddings]).reshape(-1, pipeline.dim)

# Perform t-SNE on the combined embeddings
X_embedded = TSNE(n_components=2, learning_rate='auto', perplexity=1.5, metric=config['metric'], random_state=123).fit_transform(combined_embeddings)

# Split the transformed embeddings back into queries and images
queries_2d, retrieved_2d, gt_2d = X_embedded[:len(query_embeddings)], X_embedded[len(query_embeddings):len(query_embeddings)+len(retrieved_embeddings)], X_embedded[len(query_embeddings)+len(retrieved_embeddings):]

# Plotting
fig, ax = plt.subplots(figsize=(12, 8))

radius = 5000 if config['mode'] == "img2text" else 15000

# Plot retrieved
for i, (x, y) in enumerate(retrieved_2d):
    if config['mode'] == "img2text": # retrieve text
        text = textwrap.fill(retrieval_vis_data[i][1][:textlength]+'...', width=20)
        text_box = TextArea(text, textprops=dict(color='black', size=7, ha='center', va='bottom'))
        ab = AnnotationBbox(text_box, (x, y), frameon=True, bboxprops=dict(boxstyle='round,pad=0.5', facecolor='white', edgecolor='black', alpha=0.7))
        ax.add_artist(ab)
    else:
        img = denormalize(retrieval_vis_data[i][0]).permute(1, 2, 0)
        imgbox = OffsetImage(img, zoom=0.35)
        ab = AnnotationBbox(imgbox, (x, y), frameon=True)
        ax.add_artist(ab)
    # plot a red circle around the retrieved with legend
    if i == 0:
        ax.scatter(x, y, s=100, facecolors='none', edgecolors='red', label='Retrieved')
    ax.scatter(x, y, s=radius, facecolors='none', edgecolors='red')

# Plot gt
for i, (x, y) in enumerate(gt_2d):
    if config['mode'] == "img2text": # retrieve text
        text = textwrap.fill(gt_vis_data[i][1][:textlength]+'...', width=20)
        text_box = TextArea(text, textprops=dict(color='black', size=7, ha='center', va='bottom'))
        ab = AnnotationBbox(text_box, (x, y), frameon=True, bboxprops=dict(boxstyle='round,pad=0.5', facecolor='white', edgecolor='black', alpha=0.7))
        ax.add_artist(ab)
    else:
        img = denormalize(gt_vis_data[i][0]).permute(1, 2, 0)
        ax.add_artist(AnnotationBbox(OffsetImage(img, zoom=0.35), (x, y), frameon=True))
    # plot a blue circle around the gt with legend
    if i == 0:
        ax.scatter(x, y, s=100, facecolors='none', edgecolors='green', label='Ground Truth')
    ax.scatter(x, y, s=radius, facecolors='none', edgecolors='green')

# PLot query
for i, (x, y) in enumerate(queries_2d):
    if config['mode'] == "img2text":
        img = denormalize(gt_vis_data[i][0]).permute(1, 2, 0)
        ax.add_artist(AnnotationBbox(OffsetImage(img, zoom=0.35), (x, y), frameon=True))
    else:
        text = textwrap.fill(gt_vis_data[i][1][:textlength]+'...', width=20)
        text_box = TextArea(text, textprops=dict(color='black', size=7, ha='center', va='bottom'))
        ab = AnnotationBbox(text_box, (x, y), frameon=True, bboxprops=dict(boxstyle='round,pad=0.5', facecolor='white', edgecolor='black', alpha=0.7))
        ax.add_artist(ab)

    offset = 15
    zoom = 0.05 if config['mode'] == "img2text" else 0.04
    if predictions[i][0] == labels[i]:
        #plot a green tick emoji in the right bottom corner
        img = plt.imread(imojify.get_img_path('✅'))
        ax.add_artist(AnnotationBbox(OffsetImage(img, zoom=zoom), (x+offset, y-offset), frameon=False))
    else:
        #plot a red cross emoji in the right bottom corner
        img = plt.imread(imojify.get_img_path('❌'))
        ax.add_artist(AnnotationBbox(OffsetImage(img, zoom=zoom), (x+offset, y-offset), frameon=False))
    
    # plot a blue circle around the query with legend
    if i == 0:
        ax.scatter(x, y, s=100, facecolors='none', edgecolors='blue', label='Query')
    ax.scatter(x, y, s=15000 if config['mode'] == "img2text" else 5000, facecolors='none', edgecolors='blue')

xmin = X_embedded[:, 0].min()
xmax = X_embedded[:, 0].max()
ymin = X_embedded[:, 1].min()
ymax = X_embedded[:, 1].max()
ax.set_xlim(xmin - abs(0.4 * xmin), xmax + abs(0.4 * xmax))
ax.set_ylim(ymin - abs(0.4 * ymin), ymax + abs(0.4 * ymax))
# ax.set_xlim(25,  xmax + abs(0.4 * xmax))
# ax.set_ylim(-150, 50)
plt.legend()
plt.show()